# Pru Wealth Data Test

Below we will explore a mock database of customers, policies and funds that are similar to the Pru Wealth data. The first section of this notebook will test your SQL skills, the second section will test your Python skills and visualisation skills.

The database file is called Prudential_Data.db

The three tables included in this database are:
* Customer_Table
* Policy_Table
* Fund_Table

You will need to have the following packages installed in your Python environment:
* numpy
* pandas
* sqlite3

For visualization use any package of your choice (e.g., matplotlib, seaborn, plotly).

Please complete and return the test before the deadline by emailing your copy of this notebook to: Denholm.Hesse@Prudential.co.uk. If you do not know the answer to a question feel free to use Google and Stack Overflow. This is to give you the option to teach yourself a concept from the internet to address a question which you do not know the answer to, please refrain from skipping questions. The test is meant to be completed by you only.

Good luck and have fun!

In [139]:
import numpy as np
import pandas as pd
import sqlite3

# SQL test

IMPORANT: Please make sure all questions in this section are solved using SQL by only editing the content of the variable *sqlite_query*. Please do not edit the below cell

In [140]:
# Establish DB connection
conn = sqlite3.connect('Prudential_Data.db')

Below is an example of how to query the SQLlite databse from this Jupyter notebook. Please note that SQLlite might have a slightly different syntax from the SQL you normally use. Please do not edit the below cell:

In [141]:
sqlite_query = """
SELECT COUNT(*) 
from policy_table 
"""

pd.read_sql(sqlite_query, conn)

,COUNT(*)
0,1635


The first table "Customer_Table" contains personal information about the customers. Please do not edit the below cell:

In [142]:
sqlite_query = """
SELECT * 
from Customer_Table 
limit 5
"""

pd.read_sql(sqlite_query, conn)

,index,Customer ID,Email,Gender,Age,Postcode
0,0,CO183683,NLUPPITT14@PARALLELS.COM,Male,90,None
1,1,CO314757,None,Female,90,None
2,2,CO720602,None,Female,90,None
3,3,CO34804,None,Female,90,1495-701
4,4,CO474411,None,Male,90,301288


The second table "Policy_Table" contains data relating to customer policies. The policy value is the current value of the policy. Please do not edit the below cell:

In [143]:
sqlite_query = """
SELECT * 
from Policy_Table 
limit 5
"""

pd.read_sql(sqlite_query, conn)

,index,Policy Number,Customer ID,Product Category,Product Name,Policy Value,Policy Start Date
0,0,RET100072,CO276506,RETIREMENT,RETIREMENT ACCOUNT,270033.0,2016-01-02 00:00:00
1,1,RET100931,CO831289,RETIREMENT,RETIREMENT ACCOUNT,26394.0,2016-01-04 00:00:00
2,2,RET100682,CO784249,RETIREMENT,RETIREMENT ACCOUNT,773567.0,2016-01-04 00:00:00
3,3,RET100447,CO295533,RETIREMENT,RETIREMENT ACCOUNT,35512.0,2016-01-07 00:00:00
4,4,RET100390,CO901735,RETIREMENT,RETIREMENT ACCOUNT,767580.0,2016-01-08 00:00:00


The final table "Fund_Table" contains historic data on the Policy Value (Policy_Value_Snapshot). The Policy_Value_Snapshot represents the value of the policy and the Date column is the date at which that value was recorded. The value may have changed overtime due to changes in the fund price or quanity of units the policy was invested in. Please do not edit the below cell:

In [144]:
sqlite_query = """
SELECT 
*
from Fund_Table
limit 5

"""

pd.read_sql(sqlite_query, conn)

,index,Policy Number,Customer ID,Date,Product Category,Product Name,Policy_Value_Snapshot
0,0,ANN1221,CO408564,2009-10-31 00:00:00,ANNUITY,STANDARD ANNUITY,None
1,1,ANN1221,CO408564,2009-11-30 00:00:00,ANNUITY,STANDARD ANNUITY,None
2,2,ANN1221,CO408564,2009-12-31 00:00:00,ANNUITY,STANDARD ANNUITY,None
3,3,ANN1221,CO408564,2010-01-31 00:00:00,ANNUITY,STANDARD ANNUITY,None
4,4,ANN1221,CO408564,2010-02-28 00:00:00,ANNUITY,STANDARD ANNUITY,None


If a question requires additional explanation for your answer, please add additional cells below the code. 

## Q1: What is the current average policy value split by each product name? 

In [145]:
sqlite_query = """
SELECT [Product Name],Avg([Policy Value])
from Policy_Table
Group By [Product Name]
"""

pd.read_sql(sqlite_query, conn)

,Product Name,Avg([Policy Value])
0,CORPORATE MEMBERSHIP,5697.285714
1,FOCUSSED ISA,5746.341270
2,NHS PENSION,5241.500000
3,PRU BOND,4516.389831
4,PRUDENTIAL INVESTMENT PLAN,5084.515152
5,PRUDENTIAL ISA,5161.135593
6,RETIREMENT ACCOUNT,504964.959960
7,SIPP,5335.842466
8,STANDARD ANNUITY,NaN


## Q2: How many customers opened a new policy in Q1 2017 who already had an existing pension product? 

In [146]:
sqlite_query = """
SELECT[Customer ID],[Product Name],[Policy Start Date]
from Policy_Table a
Where [Policy Start Date] Between '2017-01-01' and '2017-03-31'
and [Customer ID] IN (SELECT[Customer ID]
                        from Policy_Table b
                        Where [Product Name] = 'NHS PENSION'
                        AND a.[Policy Start Date] > b.[Policy Start Date])
"""
pd.read_sql(sqlite_query, conn)

,Customer ID,Product Name,Policy Start Date
0,CO55494,RETIREMENT ACCOUNT,2017-01-30 00:00:00
1,CO163919,RETIREMENT ACCOUNT,2017-01-30 00:00:00
2,CO760027,RETIREMENT ACCOUNT,2017-01-31 00:00:00
3,CO143604,RETIREMENT ACCOUNT,2017-02-02 00:00:00


## Q3: What was the customer ID that had the biggest difference in policy value between its earliest and latest date (for any product) ?

In [147]:
sqlite_query = """
WITH FT_CTE AS (
  SELECT [customer id] AS Customer_Id,Max([policy_value_snapshot]) - Min([policy_value_snapshot]) As PolicyVaulueDifference
  FROM Fund_Table 
  Where [policy_value_snapshot] <> 'NULL'
  Group By [customer id]
  Order by [Date]
)
SELECT Customer_Id,max(PolicyVaulueDifference) as customerwithBiggestDiff
from FT_CTE
WHERE PolicyVaulueDifference = (SELECT max(PolicyVaulueDifference) from FT_CTE)
"""

pd.read_sql(sqlite_query, conn)

,Customer_Id,customerwithBiggestDiff
0,CO517259,996856.0


# Python Test

Same data, but now let's have some fun in python. Please do not use SQL for this part of the exercise. 

Feel free to add more cells of code or text

In [148]:
Customer_Table = pd.read_sql('SELECT * FROM Customer_Table', conn)
Policy_Table = pd.read_sql('SELECT * FROM Policy_Table', conn)
Fund_Table = pd.read_sql('SELECT * FROM Fund_Table', conn)

## Q4: Produce a list of customers which have an invalid email address. What do you notice about these customers? ##

Feel free to use any packages of your choice

In [149]:
#Insert your answer below: 
import re
# Establish DB connection
#conn = sqlite3.connect('C:\SIMS8_DevOps\SIMS8_DevOps\PythonTest\Prudential_Data.db')

Customer_Table = pd.read_sql('SELECT [Customer ID],[Email] FROM Customer_Table', conn)

regex = '^(\w|\.|\_|\-)+[@](\w|\_|\-|\.)+[.]\w{2,3}$'
 
def check(email):
 
    if(email != None and re.search(regex, email)):
        return True
 
    else:
        return False

Customer_Table['is_valid_email'] = Customer_Table['Email'].apply(lambda x:check(x))

print (Customer_Table)

    Customer ID                           Email  is_valid_email
0      CO183683        NLUPPITT14@PARALLELS.COM            True
1      CO314757                            None           False
2      CO720602                            None           False
3       CO34804                            None           False
4      CO474411                            None           False
..          ...                             ...             ...
994    CO957494  CFRAMMINGHAM37@STUMBLEUPON.COM            True
995    CO503828      RRICCIARDELLI38@GOOGLE.COM            True
996    CO340813           ODYZARTMY@SAMSUNG.COM            True
997    CO403019          VENDLENT@MICROSOFT.COM            True
998    CO274837       IOFIHILLIEDJ@BLOGSPOT.COM            True

[999 rows x 3 columns]


## Q5: The Managing Director of Pru Wealth needs to see how policy value differs between other variables in the database (e.g. customer demographics) . They don't have time to read a report, so have asked for a cool visualisation that tells a story.

Visualise something cool, tell us a story! You can use packages like Matplotlib, Seaborn or Plotly.Tell us why you chose to visualise the data in this way and how it can be used from a business perspective.


In [150]:
# Insert your answer below:
from matplotlib.legend import Legend 
import matplotlib.pyplot as plt
import plotly.express as px

# Establish DB connection

#conn = sqlite3.connect('C:\SIMS8_DevOps\SIMS8_DevOps\PythonTest\Prudential_Data.db')

sqlite_query = """
SELECT CASE 
		WHEN ct.age BETWEEN 1 and 20 then '1-20'
       	WHEN ct.age BETWEEN 21 and 30 then '21-30'
        WHEN ct.age BETWEEN 31 and 40 then '31-40'
        WHEN ct.age BETWEEN 41 and 50 then '41-50'
        WHEN ct.age BETWEEN 51 and 60 then '51-60'
        WHEN ct.age BETWEEN 61 and 70 then '61-70'
        WHEN ct.age BETWEEN 71 and 80 then '71-80'
        WHEN ct.age BETWEEN 81 and 90 then '81-90'
        WHEN ct.age BETWEEN 91 and 100 then '91-100'
       END as 'Age Range',
ct.[gender] as Gender,round(Avg(pt.[Policy Value]),2) AS 'Avg Policy Value'
FROM Customer_Table ct join Policy_Table pt on ct.[customer id] = pt.[customer id]
Group by CASE 
		WHEN ct.age BETWEEN 1 and 20 then '1-20'
       	WHEN ct.age BETWEEN 21 and 30 then '21-30'
        WHEN ct.age BETWEEN 31 and 40 then '31-40'
        WHEN ct.age BETWEEN 41 and 50 then '41-50'
        WHEN ct.age BETWEEN 51 and 60 then '51-60'
        WHEN ct.age BETWEEN 61 and 70 then '61-70'
        WHEN ct.age BETWEEN 71 and 80 then '71-80'
        WHEN ct.age BETWEEN 81 and 90 then '81-90'
        WHEN ct.age BETWEEN 91 and 100 then '91-100'
       END,ct.[gender]
"""
pd1 = pd.read_sql(sqlite_query, conn)

#ns.catplot(data=pd1, kind="bar", x="Age Range", y="Avg Policy Value", hue="Gender")

x="Age Range"
y="Avg Policy Value"
color_discrete_map1 = {'Male': 'rgb(131, 153, 214)','Female': 'rgb(206, 150, 205)'}

fig = px.bar(pd1, x="Age Range", y="Avg Policy Value", color = 'Gender',
hover_data=["Age Range", "Gender","Avg Policy Value"], 
labels={"Avg Policy Value":'Avg Policy Value'}, height=500,width=1200,
color_discrete_map = color_discrete_map1)

fig.update_layout(barmode='group',
    title = "Policy Value by Age Range and Gender",
    bargap =  0.15,
    bargroupgap= 0.1,
    legend_title_text ="",
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1))

fig.show()
